# Geospatial Python
## Temporal assessment using command line arguments 
Setup: https://carpentries-incubator.github.io/geospatial-python/index.html

Based on Instruction: https://carpentries-incubator.github.io/geospatial-python/09-raster-calculations.html

Boundary data source - https://burnseverity.cr.usgs.gov/ravg/data-access, searched for "cameron peak"

This notebook does the following:
- Loads a boundary file of a fire boundary
- Searches for satellite data within the area and during the growing season for a specific year (and clips the data)
- Translates the satellite data into a value representing vegetative growth
- Classifies and generates statisitics of the vegetative growth, saves output as both text and geotiff 

We'll want to compare different years to see pre, post, and potentially other years of vegetive growth within the burn area
To do this, we'll leverage a library called papermill (https://github.com/nteract/papermill) allowing us to pass parameters to the notebook which will dynamically load, analyze, and save our results for a specific year of satellite data.

When we deploy this script to an HPC environment, running mulitple instances of it will allow us to much more rapidly get our results.

Here's one call of this script:

```
papermill Boundary\ Raster\ Classification.ipynb output_2021.ipynb -p year 2021
```

In [ ]:
import pystac
import geopandas as gpd
import folium # to make interactive maps
import rioxarray
from rioxarray import merge
from pystac_client import Client # to query STAC API endpoint

import geojson # to parse spatial data format
import folium # to create an interactive map
from folium.plugins import Draw # to allow drawing
from localtileserver import TileClient, get_folium_tile_layer # to visualize the geotif 

import numpy as np
import xarray
import earthpy.plot as ep
import matplotlib.pyplot as plt


In [ ]:
# start by setting a variable for the year
# The code cell has the tag 'parameters' (see https://github.com/nteract/papermill)
# this tag will allow us to replace the contents of the cell using the command line
# Note: The Cameron peak fire occured August 13, 2020
year = 2020

In [ ]:
# load in our boundary data
boundary = gpd.read_file("data/co4060910587920200813_20180915_20210907_burn_bndy.shp")
boundary

In [ ]:
# print(boundary.columns)
print(boundary.dtypes)

In [ ]:
# view the map boundary

boundary['Ig_Date'] = boundary['Ig_Date'].astype(str) # otherwise error "Object of type Timestamp is not JSON serializable"

map = folium.Map([boundary.iloc[1]['BurnBndLat'],boundary.iloc[1]['BurnBndLon']], zoom_start = 10)
folium.GeoJson(boundary).add_to(map)

map

In [ ]:
boundary.boundary

In [ ]:
boundary.crs

In [ ]:
boundary=boundary.to_crs("4326")

In [ ]:
poly=boundary.geometry.union_all()
poly.bounds

In [ ]:
# perform metadata search from Sentinel-2, Level 2A, to retrieve Cloud Optimized GeoTiffs (COGs)
api_url = "https://earth-search.aws.element84.com/v1"

# open the api
client = Client.open(api_url)
# store a variable pointing to the collection of interest
# Note: collection ID is taken from Sentinel-2 Level 2A - https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/collections/sentinel-2-c1-l2a
collection = "sentinel-2-l2a" 

search = client.search(
    collections=[collection],
    bbox=poly.bounds,# https://datatracker.ietf.org/doc/html/rfc7946#section-5
    datetime=str(year)+"-07-01/"+str(year)+"-07-31",
    query=["eo:cloud_cover<20"],
    limit=10
)
# show the number of scenes (i.e. the portion of the footage recorded by the satellite)
print(search.matched())

In [ ]:
#sort the items and get the first
items = search.item_collection()

items_sorted = sorted(items, key=lambda x: x.properties["eo:cloud_cover"]) # sorting and then selecting by cloud cover

In [ ]:
# lets short list the items which we'll merge later and then clip
# we only need the hrefs
red_item_hrefs=[]
nir_item_hrefs=[]

for i in items_sorted[0:3]:
    print(i)
    red_item_hrefs.append(i.assets["red"].href)
    nir_item_hrefs.append(i.assets["nir08"].href)

In [ ]:

m = folium.Map([boundary.iloc[1]['BurnBndLat'],boundary.iloc[1]['BurnBndLon']], zoom_start = 10)
folium.GeoJson(boundary).add_to(m)

# view the items on the map
for id, i in enumerate(red_item_hrefs):
    tiles = TileClient(i) # create tiles client
    tile_layer = get_folium_tile_layer(tiles, name='red_'+str(id)) # create tile layer
    tile_layer.add_to(m)

# show the bounds
folium.Rectangle(
    bounds=[[poly.bounds[1], poly.bounds[0]], [poly.bounds[3], poly.bounds[2]]],
).add_to(m)

draw = Draw(export=True)
draw.add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
# open the rasters and store in separate lists, using the argument masked=True.
red_rasters=[]
for i in red_item_hrefs:
    red_rasters.append(rioxarray.open_rasterio(i, masked=True))

nir_rasters=[]
for i in nir_item_hrefs:
    nir_rasters.append(rioxarray.open_rasterio(i, masked=True))

In [ ]:
# set out boundry to the crs of the raster
boundary_new_crs=boundary.to_crs(red_rasters[0].rio.crs)
poly_new_crs=boundary_new_crs.geometry.union_all()
poly_new_crs.bounds

In [ ]:
red_merged = merge.merge_arrays(red_rasters,poly_new_crs.bounds)
nir_merged = merge.merge_arrays(nir_rasters,poly_new_crs.bounds)

In [ ]:
red_merged.rio.to_raster("red_merged"+str(year)+".tif")
nir_merged.rio.to_raster("nir_merged"+str(year)+".tif")

## Raster Math

## Crop raster data with polygons

In [ ]:
#check the shapes of the two rasters in the following way
print(red_merged.shape, nir_merged.shape)

In [ ]:
# As their width and height do not match, use reproject_match to both reproject and clip the raster to the CRS and extent of another raster.
red_merged_matched = red_merged.rio.reproject_match(nir_merged,nodata=np.nan ) # Set NaN as NoData
print(red_merged_matched.shape)

In [ ]:
# compute the NDVI as a new raster 
ndvi = (nir_merged - red_merged_matched)/ (nir_merged + red_merged_matched)
print(ndvi)

In [ ]:
# plot the output NDVI
ndvi.plot()

In [ ]:
# plot a histogram to see the spread of values accross 50 bins
ndvi.plot.hist(bins=50)

In [ ]:
# Discretize the color plot by specifying the intervals
class_bins = (-1, 0., 0.2, 0.7, 1)
ndvi.plot(levels=class_bins)

In [ ]:
# Missing values can be interpolated from the values of neighbouring grid cells using the .interpolate_na method. 
ndvi_nonan = ndvi.interpolate_na(dim="x")

# save the output
ndvi_nonan.rio.to_raster("NDVI"+str(year)+".tif")

## Classifying Continuous Rasters in Python

Reduce the complexity of the map by classifying it. 

Classification involves assigning each pixel in the raster to a class based on its value. 

In Python, we can accomplish this using the *numpy.digitize* function

Note: by default, each class includes the left but not the right bound. This is not an issue here, since the computed range of NDVI values is fully contained in the open interval (-1; 1) (see exercise above).

In [ ]:
import numpy as np
import xarray

# Defines the bins for pixel values
class_bins = (-1, 0., 0.2, 0.7, 1)

# The numpy.digitize function returns an unlabeled array, in this case, a
# classified array without any metadata. That doesn't work--we need the
# coordinates and other spatial metadata. We can get around this by using
# "xarray.apply_ufunc", which can run the function across the data array while
# preserving metadata.
ndvi_classified = xarray.apply_ufunc(
    np.digitize,
    ndvi_nonan,
    class_bins,
    dataset_fill_value=np.nan
)

In [ ]:
# Visualize the classified NDVI, customizing the plot with proper title and legend
import earthpy.plot as ep
import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap

# Define color map of the map legend
ndvi_colors = ["blue", "gray", "green", "darkgreen"]
ndvi_cmap = ListedColormap(ndvi_colors)

# Define class names for the legend
category_names = [
    "Water",
    "No Vegetation",
    "Sparse Vegetation",
    "Dense Vegetation"
]

# We need to know in what order the legend items should be arranged
category_indices = list(range(len(category_names)))

# Make the plot
im = ndvi_classified.plot(cmap=ndvi_cmap, add_colorbar=False)
plt.title("Classified NDVI")
# earthpy helps us by drawing a legend given an existing image plot and legend items, plus indices
ep.draw_legend(im_ax=im, classes=category_indices, titles=category_names)

# Save the figure
# plt.savefig("NDVI_classified.png", bbox_inches="tight", dpi=300)

In [ ]:
# Export the classified NDVI raster object to a GeoTiff
ndvi_classified.rio.to_raster("NDVI"+str(year)+"_classified.tif", dtype="int32")

In [ ]:
ndvi_classified.plot.hist()

In [ ]:
#Load both raster datasets: NDVI.tif and NDVI_classified.tif. 
#Then, calculate zonal statistics for each class_bins. Inspect the output of the zonal_stats function.

from xrspatial import zonal_stats
stats=zonal_stats(ndvi_classified.squeeze(), ndvi.squeeze())
stats

In [ ]:
# output counts to a csv file
file_path = "output.csv"
try: 
    with open(file_path, 'x') as file: 
        file.write("year,"+",".join(category_names)+ "\n") 
except FileExistsError: 
    pass



In [ ]:
 with open(file_path, 'a') as file: 
        file.write(str(year)+","+",".join(str(x) for x in stats["count"])+ "\n") 